In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import statsmodels.api as sm
import pickle
import sys
import os
import glob
import output
import random
import DEMENTpy
import datetime
from dssat import *
import dssat_setup as ds
import dssat_model as dm
import dssat_output as do
from dssat.models import *
from dssat.utils import *
import comtypes.client as cc

dssat = cc.GetActiveObject("DSSAT.Application")
ceres_drought = dssat.Component("CERES-Maize")
exp_file_path = "path/to/experiment/file"
ceres_drought.Open(exp_file_path)
weather_input = ceres_drought.Inputs.Weather

#set the weather data using the provided dates and rainfall data
def set_weather_data(dates, rainfall):
    num_records = len(dates)
    weather_input.Nval = num_records
    
    for i in range(num_records):
        date_str = dates[i].strftime("%Y,%m,%d")
        weather_input.Date[i] = date_str
        weather_input.Rain[i] = rainfall[i]

set_weather_data(dates, rainfall)
ceres_drought.Run()
ceres_drought.Close()

#VULNERABILITY MODEL

soil_data = {
    'soil_type': 'sandy loam',
    'soil_texture': 'SAL',
    'ph': 6.0,
    'cec': 10.0,
    'organic_matter': 1.5,
}

weather_data = {
    'station_id': 'STN01',
    'latitude': 40.0,
    'longitude': -80.0,
    'elevation': 200.0,
    'year': 2022,
    'weather_file': 'STN01.WTH',
}

simulation_parameters = {
    'crop': 'maize',
    'planting_date': datetime.datetime(2022, 5, 1),
    'harvest_date': datetime.datetime(2022, 10, 1),
    'fertilizer_rate': 200,
}

#use DEMENTpy to add additional soil parameters
def get_pickled_data(key):
    
    datalist = []

    ## single run
    #filelist = glob.glob(key+'_'+'20201'+'.pickle')
    filelist_19   = glob.glob(key + '_' + '2020' + '[1-9].pickle')
    filelist_1019 = glob.glob(key + '_' + '2020' + '1[0-9].pickle')
    filelist_2020 = glob.glob(key + '_' + '2020' + '20.pickle')
    filelist = filelist_19 + filelist_1019 + filelist_2020

    filelist.sort(reverse=False)

    for file in filelist:
        with open(file,"rb") as f:
            data = pickle.load(f)
        datalist.append(data)

    return filelist, datalist


def community_drought(data):
    """
    Calculate community-level drought tolerance
    """

    Relative_mass = data.MicrobesSeries.div(data.MicrobesSeries.sum(axis=0),axis=1)
    # enzyme_trait  = data.Microbial_traits['Enz_Induci_Cost'] * data.Microbial_traits['Enz_Gene']
    #enzyme_trait  = (data.Microbial_traits['Enz_Induci_Cost'] + data.Microbial_traits['Enz_Consti_Cost']) * data.Microbial_traits['Enz_Gene']
    drought_tol   = data.Microbial_traits['Drought_tolerance']
    community_drought = Relative_mass.mul(drought_tol,axis=0).sum(axis=0)

    return community_drought

site = sys.argv[1]    # base site name
key = sys.argv[2]     # target site

os.chdir('../output_'+site)

#call function get_pickled_data()
filelist, datalist = get_pickled_data(key)

#call function community_drought()
drought_tol = pd.concat([community_drought(data) for data in datalist], axis=1, sort=False)

drought_tol.to_csv('data/' + 'Drt_' + site +'_'+ key + '.csv')

soil_microbiome_health = pd.read_csv("drought_tol")

#Define the path to the CSV file containing additional soil parameters
soil_data_file = 'drought_tol'
soil_df = pd.read_csv(soil_data_file)

soil_type = soil_df['soil_type'][0]
soil_texture = soil_df['soil_texture'][0]
ph = soil_df['ph'][0]
cec = soil_df['cec'][0]
organic_matter = soil_df['organic_matter'][0]

#Add the additional soil parameters to the soil parameters dictionary
soil_parameters = {
    'data.Microbial_traits': organic_matter
    # Add any additional soil properties required by DSSAT-CENTURY
}

#Set up DSSAT-CENTURY simulation
dssat_dir = 'C:/DSSAT47'
crop_dir = os.path.join(dssat_dir, 'MAIZE')
soil_dir = os.path.join(dssat_dir, 'SOILS')
weather_dir = os.path.join(dssat_dir, 'WEATHER')
output_dir = 'C:/DSSAT47/OUTPUTS'

ds.setup_dssat_directories(dssat_dir, crop_dir, soil_dir, weather_dir, output_dir)
sim_dir = ds.setup_simulation_directory(dssat_dir, crop_dir, soil_dir, weather_dir, output_dir, simulation_parameters['crop'], simulation_parameters['planting_date'])

#Define the maize field simulation model
def simulate_maize_field(biodiversity, stress_level, nitrogen_level):

    d = DSSAT("maize")
    d.set_weather("weather.dat")
    
    #Set simulation parameters from predefined dataframes. You can also do this by importing a subprocess if this is already formatted in a system file.
    d.set_control(["PHOTOPERIOD         AUTO   ",

                   "PLANTING            2001001",

                   "EMERGENCE           2001002",

                   "HARVEST             2001101",

                   "IRRIGATION          NO",

                   "FERTIGATION         NO",

                   "CULTIVAR            99MAIZE",

                   "NITROGEN            " + str(nitrogen_level),

                   "TEMPERATURE UNITS   C     ",

                   "SOLAR UNITS         MJ m^-2 day^-1",

                   "WIND UNITS          m s^-1",

                   "PRESSURE UNITS      kPa   ",

                   "TIME STEP           1"])

    d.set_initial_conditions({"PLANTING DATE": "01/01/2001",

                               "CULTIVAR": "99MAIZE",

                               "NITROGEN": nitrogen_level,

                               "SOIL": "MOLLISOLS"})
    
    d.set_weather("weather.dat")
    d.set_soil("soil.dat")
    d.set_management("management.dat")

    #Set soil microbiodiversity parameters
    d.set_parameter(organic_matter.dat)
    
#Run DSSAT-CENTURY simulation
dm.run_dssat_model(dssat_dir, sim_dir)
output_data = do.read_dssat_output(output_dir, simulation_parameters['maize'])

df = pd.read_dssat_output(output_file_path, delimiter="\t")
dates = pd.to_datetime(df["Date"], format="%Y-%m-%d")
rainfall = df["maize"]
dssat_interface.set_weather_data(dates, rainfall)

model = CERESMaize()
output = model.run_weather_soil_management(weather, soil, management)